# User Actions and State Transition FUnctions - (pt 3 of 4)

In [3]:
"""Imports or defines most simulator configuration variables"""

import decimal
from typing import TypeVar
from bancor_research.bancor_simulator.v3.spec.network import *

decimal.getcontext().prec = len(str(2 ** 512 - 1))
decimal.getcontext().rounding = decimal.ROUND_HALF_EVEN

Decimal = decimal.Decimal
DataFrame = pandas.DataFrame
PandasDataFrame = TypeVar("pandas.core.frame.DataFrame")

SECONDS_PER_DAY = 24 * 60 * 60
TIMESTAMP = 0
DECIMALS = 18
TRADING_FEE    = "1%"
NETWORK_FEE    = "20%"
WITHDRAWAL_FEE = "0.25%"
BNT_FUNDING_LIMIT = "1000000"
BNT_MIN_LIQUIDITY = "10000"
EP_VAULT_BALANCE  = "0"
COOLDOWN_TIME  = 7 * SECONDS_PER_DAY
NUM_TIMESTAMPS = 30 * SECONDS_PER_DAY
PRICE_FEEDS_PATH = "https://bancorml.s3.us-east-2.amazonaws.com/price_feeds.parquet"
PRICE_FEEDS = DataFrame({
    "INDX": (    0.00 for _ in range(NUM_TIMESTAMPS)),
    "vbnt": (    1.00 for _ in range(NUM_TIMESTAMPS)),
    "tkn" : (    2.50 for _ in range(NUM_TIMESTAMPS)),
    "bnt" : (    2.50 for _ in range(NUM_TIMESTAMPS)),
    "link": (   15.00 for _ in range(NUM_TIMESTAMPS)),
    "eth" : ( 2500.00 for _ in range(NUM_TIMESTAMPS)),
    "wbtc": (40000.00 for _ in range(NUM_TIMESTAMPS)),
})
USERS = [
    "Alice",
    "Bob",
    "Charlie",
    "Trader",
]
TOKENS = [
    "eth",
    "link",
    "tkn",
    "wbtc",
]
WHITELIST = {
    TOKEN : {
        "decimals": DECIMALS,
        "trading_fee": TRADING_FEE,
        "bnt_funding_limit": BNT_FUNDING_LIMIT,
        "ep_vault_balance": EP_VAULT_BALANCE,
    }
    for TOKEN in TOKENS
}

def read_price_feeds(price_feeds_path: str):
    price_feeds = pandas.read_parquet(price_feeds_path)
    price_feeds.columns = [col.lower() for col in price_feeds.columns]
    return price_feeds

# Initialize the main simulator class
v3 = BancorDapp(price_feeds=PRICE_FEEDS,network_fee='0.15')

# Set BNT Minimum Liquidity
v3.global_state.set_bnt_min_liquidity(tkn_name='tkn', value='1')
v3.global_state.set_bnt_min_liquidity(tkn_name='bnt', value='1')

# Set TKN Trading Liquidity
v3.global_state.set_bnt_trading_liquidity(tkn_name='tkn', value='100')
v3.global_state.set_tkn_trading_liquidity(tkn_name='tkn', value='100')

# Set TKN Vault Balance
v3.global_state.set_vault_balance(tkn_name='tkn', value='200')

# Set Trading Fees for BNT
v3.global_state.set_trading_fee(tkn_name='bnt', value='0.002')

# Set Trading Fees for TKN
v3.global_state.set_trading_fee(tkn_name='tkn', value='0.002')

# Create the users
v3.create_user('Alice')

# Set the initial starting balances
v3.set_user_balance(user_name='Alice', tkn_name='bnt',tkn_amt=1000000)
v3.set_user_balance(user_name='Alice', tkn_name='tkn',tkn_amt=1000000)

v3.enable_trading('tkn')
v3.enable_trading('bnt')

v3.describe()

bnt eth link      tkn wbtc bnbnt  \
1 Account Alice                     1000000   0    0  1000000    0     0   
2 Pool    a: TKN Staked Balance           0   0    0        0    0     0   
          b: TKN Trading Liquidity        0   0    0        2    0     0   
          c: BNT Trading Liquidity        0   0    0        2    0     0   
          d: BNT Current Funding          0   0    0        2    0     0   
          e: Spot Rate                    0   0    0        1    0     0   
          f: Average Rate                 0   0    0        1    0     0   
          g: Average Inverse Rate         0   0    0        1    0     0   
3 Network BNT Staked Balance              4   0    0        0    0     0   
          Master Vault                    4   0    0      200    0     0   
          Protection Vault                0   0    0        0    0     0   
          Protocol Equity                 0   0    0        0    0     4   
          Rewards Vault                   0   0    0        0    0     0   
          Vortex                          0   0    0        0    0     0   

                                   bneth bnlink bntkn bnwbtc vbnt  
1 Account Alice                        0      0     0      0    0  
2 Pool    a: TKN Staked Balance        0      0     0      0    0  
          b: TKN Trading Liquidity     0      0     0      0    0  
          c: BNT Trading Liquidity     0      0     0      0    0  
          d: BNT Current Funding       0      0     0      0    0  
          e: Spot Rate                 0      0     0      0    0  
          f: Average Rate              0      0     0      0    0  
          g: Average Inverse Rate      0      0     0      0    0  
3 Network BNT Staked Balance           0      0     0      0    0  
          Master Vault                 0      0     0      0    0  
          Protection Vault             0      0     0      0    0  
          Protocol Equity              0      0     0      0    0  
          Rewards Vault                0      0     0      0    0  
          Vortex                       0      0     0      0    0

#### Review from last session

Configure the system state.


In [14]:
import pandas as pd

# Set the price feeds
price_feeds = pd.DataFrame({
        "INDX": (    0.00 for _ in range(100)),
        "vbnt": (    1.00 for _ in range(100)),
        "eth" : (    1.00 for _ in range(100)),
        "link" : (   1.00 for _ in range(100)),
        "wbtc" : (   1.00 for _ in range(100)),
        "tkn" : (   1.00  for _ in range(100)),
        "bnt" : (   1.0 for _ in range(100)),
    })

In [20]:
# Initialize the main simulator class
v3 = BancorDapp(price_feeds=price_feeds,network_fee='0.15')

# Set BNT Minimum Liquidity
v3.global_state.set_bnt_min_liquidity(tkn_name='tkn', value='1')
v3.global_state.set_bnt_min_liquidity(tkn_name='bnt', value='1')

# Set TKN Trading Liquidity
v3.global_state.set_bnt_trading_liquidity(tkn_name='tkn', value='100')
v3.global_state.set_tkn_trading_liquidity(tkn_name='tkn', value='100')

# Set TKN Vault Balance
v3.global_state.set_vault_balance(tkn_name='tkn', value='200')

# Set Trading Fees for BNT
v3.global_state.set_trading_fee(tkn_name='bnt', value='0.002')

# Set Trading Fees for TKN
v3.global_state.set_trading_fee(tkn_name='tkn', value='0.002')

# Create the users
v3.create_user('Alice')

# Set the initial starting balances
v3.set_user_balance(user_name='Alice', tkn_name='bnt',tkn_amt=1000000)
v3.set_user_balance(user_name='Alice', tkn_name='tkn',tkn_amt=1000000)

v3.enable_trading('tkn')
v3.enable_trading('bnt')

v3.describe()

bnt eth link      tkn wbtc bnbnt  \
1 Account Alice                     1000000   0    0  1000000    0     0   
2 Pool    a: TKN Staked Balance           0   0    0        0    0     0   
          b: TKN Trading Liquidity        0   0    0        2    0     0   
          c: BNT Trading Liquidity        0   0    0        2    0     0   
          d: BNT Current Funding          0   0    0        2    0     0   
          e: Spot Rate                    0   0    0        1    0     0   
          f: Average Rate                 0   0    0        1    0     0   
          g: Average Inverse Rate         0   0    0        1    0     0   
3 Network BNT Staked Balance              4   0    0        0    0     0   
          Master Vault                    4   0    0      200    0     0   
          Protection Vault                0   0    0        0    0     0   
          Protocol Equity                 0   0    0        0    0     4   
          Rewards Vault                   0   0    0        0    0     0   
          Vortex                          0   0    0        0    0     0   

                                   bneth bnlink bntkn bnwbtc vbnt  
1 Account Alice                        0      0     0      0    0  
2 Pool    a: TKN Staked Balance        0      0     0      0    0  
          b: TKN Trading Liquidity     0      0     0      0    0  
          c: BNT Trading Liquidity     0      0     0      0    0  
          d: BNT Current Funding       0      0     0      0    0  
          e: Spot Rate                 0      0     0      0    0  
          f: Average Rate              0      0     0      0    0  
          g: Average Inverse Rate      0      0     0      0    0  
3 Network BNT Staked Balance           0      0     0      0    0  
          Master Vault                 0      0     0      0    0  
          Protection Vault             0      0     0      0    0  
          Protocol Equity              0      0     0      0    0  
          Rewards Vault                0      0     0      0    0  
          Vortex                       0      0     0      0    0

In [21]:
timestamp = 1
v3.trade('10000', 'bnt','tkn', 'Alice', timestamp=timestamp)
v3.describe()

bnt  \
1 Account Alice                                                                990000   
2 Pool    a: TKN Staked Balance                                                     0   
          b: TKN Trading Liquidity                                                  0   
          c: BNT Trading Liquidity                                                  0   
          d: BNT Current Funding                                                    0   
          e: Spot Rate                                                              0   
          f: Average Rate                                                           0   
          g: Average Inverse Rate                                                   0   
3 Network BNT Staked Balance                                                        4   
          Master Vault                                                          10004   
          Protection Vault                                                          0   
          Protocol Equity                                                           0   
          Rewards Vault                                                             0   
          Vortex                    9.09272727272727272727272727272727272727272727...   

                                   eth link  \
1 Account Alice                      0    0   
2 Pool    a: TKN Staked Balance      0    0   
          b: TKN Trading Liquidity   0    0   
          c: BNT Trading Liquidity   0    0   
          d: BNT Current Funding     0    0   
          e: Spot Rate               0    0   
          f: Average Rate            0    0   
          g: Average Inverse Rate    0    0   
3 Network BNT Staked Balance         0    0   
          Master Vault               0    0   
          Protection Vault           0    0   
          Protocol Equity            0    0   
          Rewards Vault              0    0   
          Vortex                     0    0   

                                                                                  tkn  \
1 Account Alice                     1000001.99560087982403519296140771845630873825...   
2 Pool    a: TKN Staked Balance     0.00399520095980803839232153569286142771445710...   
          b: TKN Trading Liquidity  0.00439912017596480703859228154369126174765046...   
          c: BNT Trading Liquidity  9992.90727272727272727272727272727272727272727...   
          d: BNT Current Funding                                                    2   
          e: Spot Rate              2271569.51231404958677685950413223140495867768...   
          f: Average Rate                                                         1.0   
          g: Average Inverse Rate                                                 1.0   
3 Network BNT Staked Balance                                                        0   
          Master Vault              198.004399120175964807038592281543691261747650...   
          Protection Vault                                                          0   
          Protocol Equity                                                           0   
          Rewards Vault                                                             0   
          Vortex                                                                    0   

                                   wbtc bnbnt bneth bnlink bntkn bnwbtc vbnt  
1 Account Alice                       0     0     0      0     0      0    0  
2 Pool    a: TKN Staked Balance       0     0     0      0     0      0    0  
          b: TKN Trading Liquidity    0     0     0      0     0      0    0  
          c: BNT Trading Liquidity    0     0     0      0     0      0    0  
          d: BNT Current Funding      0     0     0      0     0      0    0  
          e: Spot Rate                0     0     0      0     0      0    0  
          f: Average Rate             0     0     0      0     0      0    0  
          g: Average Inverse Rate     0     0     0      0     0      0 

In [22]:
timestamp = 2
v3.trade('1.470748019531512', 'tkn','bnt', 'Alice', timestamp=timestamp)
v3.describe()

bnt  \
1 Account Alice                     999943.180641947920403643986770117878282927644...   
2 Pool    a: TKN Staked Balance                                                     0   
          b: TKN Trading Liquidity                                                  0   
          c: BNT Trading Liquidity                                                  0   
          d: BNT Current Funding                                                    0   
          e: Spot Rate                                                              0   
          f: Average Rate                                                           0   
          g: Average Inverse Rate                                                   0   
3 Network BNT Staked Balance        23.9062874976071592850507871409774757608110561...   
          Master Vault              60.8193580520795963560132298821217170723553494...   
          Protection Vault                                                          0   
          Protocol Equity                                                           0   
          Rewards Vault                                                             0   
          Vortex                    9.11265348643859120503554087346899192222849409...   

                                   eth link  \
1 Account Alice                      0    0   
2 Pool    a: TKN Staked Balance      0    0   
          b: TKN Trading Liquidity   0    0   
          c: BNT Trading Liquidity   0    0   
          d: BNT Current Funding     0    0   
          e: Spot Rate               0    0   
          f: Average Rate            0    0   
          g: Average Inverse Rate    0    0   
3 Network BNT Staked Balance         0    0   
          Master Vault               0    0   
          Protection Vault           0    0   
          Protocol Equity            0    0   
          Rewards Vault              0    0   
          Vortex                     0    0   

                                                                                  tkn  \
1 Account Alice                     1000000.52485286029252319296140771845630873825...   
2 Pool    a: TKN Staked Balance     0.00399520095980803839232153569286142771445710...   
          b: TKN Trading Liquidity  1.47514713970747680703859228154369126174765046...   
          c: BNT Trading Liquidity  49.7067045656410051509776890086527251501268553...   
          d: BNT Current Funding    21.9062874976071592850507871409774757608110561...   
          e: Spot Rate              33.6960993433495018817167481077645463912147366...   
          f: Average Rate           454314.702462809917355371900826446280991735537...   
          g: Average Inverse Rate   0.80000008804485133112208754703126159227459189...   
3 Network BNT Staked Balance                                                        0   
          Master Vault              199.475147139707476807038592281543691261747650...   
          Protection Vault                                                          0   
          Protocol Equity                                                           0   
          Rewards Vault                                                             0   
          Vortex                                                                    0   

                                   wbtc bnbnt bneth bnlink bntkn bnwbtc vbnt  
1 Account Alice                       0     0     0      0     0      0    0  
2 Pool    a: TKN Staked Balance       0     0     0      0     0      0    0  
          b: TKN Trading Liquidity    0     0     0      0     0      0    0  
          c: BNT Trading Liquidity    0     0     0      0     0      0    0  
          d: BNT Current Funding      0     0     0      0     0      0    0  
          e: Spot Rate                0     0     0      0     0      0    0  
          f: Average Rate             0     0     0      0     0      0    0  
          g: Average Inverse Rate     0     0     0      0     0      0 

In [23]:
timestamp = 2
v3.deposit('tkn', '1', 'Alice', timestamp=timestamp)
v3.describe()

bnt  \
1 Account Alice                     999943.180641947920403643986770117878282927644...   
2 Pool    a: TKN Staked Balance                                                     0   
          b: TKN Trading Liquidity                                                  0   
          c: BNT Trading Liquidity                                                  0   
          d: BNT Current Funding                                                    0   
          e: Spot Rate                                                              0   
          f: Average Rate                                                           0   
          g: Average Inverse Rate                                                   0   
3 Network BNT Staked Balance        2.00000000000000000000000000000000000000000000...   
          Master Vault              11.1126534864385912050355408734689919222284940...   
          Protection Vault                                                          0   
          Protocol Equity                                                           0   
          Rewards Vault                                                             0   
          Vortex                    9.11265348643859120503554087346899192222849409...   

                                   eth link  \
1 Account Alice                      0    0   
2 Pool    a: TKN Staked Balance      0    0   
          b: TKN Trading Liquidity   0    0   
          c: BNT Trading Liquidity   0    0   
          d: BNT Current Funding     0    0   
          e: Spot Rate               0    0   
          f: Average Rate            0    0   
          g: Average Inverse Rate    0    0   
3 Network BNT Staked Balance         0    0   
          Master Vault               0    0   
          Protection Vault           0    0   
          Protocol Equity            0    0   
          Rewards Vault              0    0   
          Vortex                     0    0   

                                                                                  tkn  \
1 Account Alice                     999999.524852860292523192961407718456308738252...   
2 Pool    a: TKN Staked Balance                                                     1   
          b: TKN Trading Liquidity                                                  0   
          c: BNT Trading Liquidity                                                  0   
          d: BNT Current Funding                                               0E-153   
          e: Spot Rate                                                              0   
          f: Average Rate                                                           0   
          g: Average Inverse Rate                                                   0   
3 Network BNT Staked Balance                                                        0   
          Master Vault              200.475147139707476807038592281543691261747650...   
          Protection Vault                                                          0   
          Protocol Equity                                                           0   
          Rewards Vault                                                             0   
          Vortex                                                                    0   

                                   wbtc  \
1 Account Alice                       0   
2 Pool    a: TKN Staked Balance       0   
          b: TKN Trading Liquidity    0   
          c: BNT Trading Liquidity    0   
          d: BNT Current Funding      0   
          e: Spot Rate                0   
          f: Average Rate             0   
          g: Average Inverse Rate     0   
3 Network BNT Staked Balance          0   
          Master Vault                0   
          Protection Vault            0   
          Protocol Equity             0   
          Rewards Vault               0   
          Vortex                      0   

                                                          

In [15]:
def trade_tkn_to_ema(
    bnt_trading_liquidity: Decimal,
    tkn_trading_liquidity: Decimal,
    trading_fee: Decimal,
    network_fee: Decimal,
    future_ema: Decimal,
) -> Decimal:
    """
    Outputs the tkn_amt that should be traded to force the ema and the spot price together on a given pool.
    """
    a = bnt_trading_liquidity
    b = tkn_trading_liquidity
    d = trading_fee
    e = network_fee
    f = future_ema
    tkn_amt = (
        (a * d * (Decimal("1") - e) - Decimal("2") * f * b)
        + (
            a
            * (
                Decimal("4") * f * b * (Decimal("1") - d * (Decimal("1") - e))
                + a * d ** Decimal("2") * (Decimal("1") - e) ** Decimal("2")
            )
        )
        ** (Decimal("1") / Decimal("2"))
    ) / (Decimal("2") * f)
    return tkn_amt

In [16]:
trade_tkn_to_ema(
    bnt_trading_liquidity=Decimal(49.706),
    tkn_trading_liquidity= Decimal(1.475),
    trading_fee=Decimal(0.002),
    network_fee=Decimal(0.15),
    future_ema=Decimal(1)
)

Decimal('7.1225699231246748329617447342171579753177889174717378401709072070704645909890230309698635626305700984396898276630836125587243564077237788215383304579146745')

In [ ]:
def trade_bnt_to_ema(
    bnt_trading_liquidity,
    tkn_trading_liquidity,
    trading_fee,
    network_fee,
    future_ema,
):
    """
    Analyze the state of any pool and create a swap that drives the ema and the spot price together.
    """

    a = bnt_trading_liquidity
    b = tkn_trading_liquidity
    d = trading_fee
    e = network_fee
    f = future_ema
    x = (
        -Decimal("2") * a
        + b * d * f
        + (
            (Decimal("2") * a - b * d * f) ** Decimal("2")
            - Decimal("4") * a * (a - b * f)
        )
        ** (Decimal("1") / Decimal("2"))
    ) / Decimal("2")
    a_recursion = (
        a * (a + x) + d * (Decimal("1") - e) * (a * x + x ** Decimal("2"))
    ) / (a + d * x)
    b_recursion = b * (a + d * x) / (a + x)
    n = 0
    p = Decimal("0.001")
    while a_recursion / b_recursion < f:
        n += 1
        p += Decimal("0.0001")
        x += x * (f**p - (a_recursion / b_recursion) ** p) / f
        a_recursion = (
            a * (a + x) + d * (Decimal("1") - e) * (a * x + x ** Decimal("2"))
        ) / (a + d * x)
        b_recursion = b * (a + d * x) / (a + x)
        if n > 20000:
            break
    bnt_amt = x
    return bnt_amt